<a href="https://colab.research.google.com/github/un1u3/ml-labs/blob/main/Deep%20Learning/Pytorch/06_Building_an_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
torch.manual_seed(42)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
df = pd.read_csv('fmnist_small.csv')

In [ ]:
df

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,...,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,213,203,174,151,188,10,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,1,0,0,0,0,0,0,0,0,0,...,69,12,0,0,0,0,0,0,0,0
5996,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5997,8,0,0,0,0,0,0,0,0,0,...,39,47,2,0,0,29,0,0,0,0
5998,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# train test split
X = df.iloc[:,1:].values
y = df.iloc[:,0].values

In [ ]:
X_train,X_test, y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
# scaling the features
X_train = X_train/255.0
X_test = X_test/ 255.0

In [ ]:
# create a Custom Dataset class
class CustomDataset(Dataset):
  def __init__(self,features,labels):
    self.features = torch.tensor(features,dtype=torch.float32)
    self.labels = torch.tensor(labels,dtype=torch.long)

  def __len__(self):
    return len(self.features)


  def __getitem__(self,index):
    return self.features[index], self.labels[index]

In [ ]:
# create train dataset object
train_dataset = CustomDataset(X_train,y_train)

In [ ]:
train_dataset[5]

(tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3608,
         0.7765, 0.5765, 0.2510, 0.1529, 0.1412, 0.2157, 0.3412, 0.4118, 0.6588,
         0.6471, 0.1882, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3765, 0.9922,
         0.8588, 0.8314, 0.9725, 1.0000, 1.0000, 1.0000, 0.8118, 0.8471, 0.9961,
         0.8471, 0.8275, 0.8627, 0.9373, 0.1490, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1490, 0.8706,
         0.8078, 0.7922, 0.7804, 0.7686, 0.7451, 0.7608, 0.8000, 0.7373, 0.7490,
         0.7451, 0.7647, 0.7922, 0.7843, 0.8353, 0.8039, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.6314,
         0.8667, 0.7961, 0.8118, 0.8039, 0.8078, 0.8000, 0.7922, 0.7922, 0.8157,
         0.8275, 0.8275, 0.8196, 0.7804, 0.8118, 0.7804, 0.8784, 0.4941, 0.0000,
         0.0000, 0.0000, 0.0

In [ ]:
# create test dataser

In [ ]:
test_dataset =  CustomDataset(X_test,y_test)

In [ ]:
# creatte train and test loader
train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=32,shuffle=False)

In [ ]:
# define nn class

class MyNN(nn.Module):

  def __init__(self, num_features):

    super().__init__()
    self.model =nn.Sequential(
    nn.Linear(num_features, 128),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(64, 10)
)


  def forward(self,x):
    return self.model(x)

In [ ]:
# set learning rate and epochs
epochs = 100
learning_rate = 0.1

In [ ]:
# instatiate the model
model = MyNN(X_train.shape[1])
model = model.to(device)

# loss function
criterion = nn.CrossEntropyLoss()

# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# training loop

for epoch in range(epochs):

  total_epoch_loss = 0
  for batch_features, batch_labels in train_loader:
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)  # CORRECT

    # forward pass
    outputs = model(batch_features)

    # calculate loss
    loss = criterion(outputs, batch_labels)

    # back pass
    optimizer.zero_grad()
    loss.backward()

    # update grads
    optimizer.step()

    total_epoch_loss = total_epoch_loss + loss.item()

  avg_loss = total_epoch_loss/len(train_loader)
  print(f'Epoch: {epoch + 1} , Loss: {avg_loss}')


Epoch: 1 , Loss: 1.3157534539699554
Epoch: 2 , Loss: 0.8037159093221029
Epoch: 3 , Loss: 0.6966404658555985
Epoch: 4 , Loss: 0.6312894382079443
Epoch: 5 , Loss: 0.5675454409917196
Epoch: 6 , Loss: 0.5317811971902847
Epoch: 7 , Loss: 0.504631938735644
Epoch: 8 , Loss: 0.48458846390247345
Epoch: 9 , Loss: 0.4574578260382017
Epoch: 10 , Loss: 0.43641685287157694
Epoch: 11 , Loss: 0.4180301856994629
Epoch: 12 , Loss: 0.40990422397851944
Epoch: 13 , Loss: 0.38798590590556464
Epoch: 14 , Loss: 0.3878264424701532
Epoch: 15 , Loss: 0.3871489401658376
Epoch: 16 , Loss: 0.35482682794332504
Epoch: 17 , Loss: 0.36844957987467447
Epoch: 18 , Loss: 0.339476534674565
Epoch: 19 , Loss: 0.33159278695782024
Epoch: 20 , Loss: 0.3341036942601204
Epoch: 21 , Loss: 0.33280915608008704
Epoch: 22 , Loss: 0.31517756745219233
Epoch: 23 , Loss: 0.3012944216777881
Epoch: 24 , Loss: 0.29022520656387013
Epoch: 25 , Loss: 0.30046984578172364
Epoch: 26 , Loss: 0.287947835624218
Epoch: 27 , Loss: 0.28060384579002856
E

In [ ]:
# set model to eval mode

model.eval()


MyNN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [ ]:
# write code to evaluaemide

In [ ]:
# evaluate the model
with torch.no_grad():
    correct = 0
    total = 0
    total_test_loss = 0
    for features, labels in test_loader:
      # move data
      batch_features, batch_labels = features.to(device), labels.to(device)

      outputs = model(batch_features)
      loss = criterion(outputs, batch_labels)
      total_test_loss += loss.item()
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == batch_labels).sum().item()

    avg_test_loss = total_test_loss / len(test_loader)
    accuracy = 100 * correct / total

    print(f'Test Loss: {avg_test_loss:.4f}')
    print(f'Test Accuracy: {accuracy:.2f}%')

Test Loss: 0.7669
Test Accuracy: 83.83%


In [ ]:
# evaluate the model
with torch.no_grad():
    correct = 0
    total = 0
    total_test_loss = 0
    for features, labels in train_loader:
      # move data
      batch_features, batch_labels = features.to(device), labels.to(device)

      outputs = model(batch_features)
      loss = criterion(outputs, batch_labels)
      total_test_loss += loss.item()
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == batch_labels).sum().item()

    avg_test_loss = total_test_loss / len(train_loader)
    accuracy = 100 * correct / total

    print(f'train Loss: {avg_test_loss:.4f}')
    print(f'train Accuracy: {accuracy:.2f}%')

train Loss: 0.0552
train Accuracy: 98.17%


# here you see overfitting
1. solutions of Overfitting
2. reducing the complexity of NN architecture
3. Regularization
4. Dropouts
5. Data Augumentation
6. Batch Normalization
7. Early Stopping

In [ ]:
# dd